In [1]:
# /Users/arshiabansal/Desktop/product-1211-sales-order-data.csv

In [2]:
import numpy as np
import pandas as pd

def tsl_policy_userinput(
    sales_order_file_path,
    initial_inventory=1000,
    desired_service_level=0.95,
    max_shelf_space_for_SKU=124400,
    MOQ=1000,
    today_date='2021-04-01',
    lead_time=4,
    total_days=100,
    review_period=7,
    max_stock_level=2000
):
    # Read the sales order data
    demand_data = pd.read_csv(sales_order_file_path, index_col=0, parse_dates=True)

    # Rename the column if necessary
    if 'total qty' in demand_data.columns:
        demand_data.rename(columns={'total qty': 'total_qty'}, inplace=True)

    # Convert 'total_qty' to numeric and fill missing values
    demand_data['total_qty'] = pd.to_numeric(demand_data['total_qty'], errors='coerce').fillna(0)

    # Convert today's date to datetime
    today_date = pd.to_datetime(today_date)

    if MOQ > max_shelf_space_for_SKU:
        raise ValueError("This order cannot be placed since the MOQ is greater than shelf space available")

    # Function to calculate safety stock
    def safety_stock_calculation(demand_std_dev, lead_time, Z):
        Z_score = abs(np.percentile(np.random.normal(size=1000000), 100 * Z))
        return Z_score * (lead_time ** 0.5) * demand_std_dev

    demand_std_dev = demand_data['total_qty'].std()
    safety_stock = safety_stock_calculation(demand_std_dev, lead_time, desired_service_level)
    print("Safety Stock:", safety_stock)

    # Forecasted demand over lead time
    def forecasted_demand_over_lead_time(demand_data, today_date, lead_time):
        todays_index = demand_data.index.get_loc(today_date)
        start_index = todays_index + 1
        end_index = todays_index + lead_time
        lead_time_df = demand_data.iloc[start_index:end_index]
        return lead_time_df['total_qty'].sum()

    dd_over_lead_time = forecasted_demand_over_lead_time(demand_data, today_date, lead_time)

    # Forecasted demand till next review period
    def forecasted_demand_till_next_review(demand_data, today_date, review_period):
        todays_index = demand_data.index.get_loc(today_date)
        start_index = todays_index
        end_index = todays_index + review_period
        review_period_df = demand_data.iloc[start_index:end_index]
        return review_period_df['total_qty'].sum()

    dd_till_next_review = forecasted_demand_till_next_review(demand_data, today_date, review_period)

    # Function to calculate reorder point (safety stock + forecasted demand till next review period + forecasted demand over lead time)
    def reorder_point_calculation(safety_stock, dd_over_lead_time, dd_till_next_review):
        return safety_stock + dd_over_lead_time + dd_till_next_review

    reorder_point = reorder_point_calculation(safety_stock, dd_over_lead_time, dd_till_next_review)
    print("Reorder Point:", reorder_point)

    # Inventory System Simulation
    try:
        todays_index = demand_data.index.get_loc(today_date)
    except KeyError:
        raise ValueError(f"Date {today_date} not found in demand data index.")
    
    end_index = todays_index + total_days

    # Slice the demand data for the simulation period
    forecasted_demand = demand_data.iloc[todays_index:end_index]['total_qty'].values

    # Generate the days for the simulation period
    days = pd.date_range(start=today_date, periods=total_days)

    # Initialize the DataFrame for the simulation
    data = {
        'day_tracker': days,
        'stock_start': np.zeros(total_days),
        'forecasted_demand': forecasted_demand,
        'stock_end': np.zeros(total_days),
        'inventory_check': [False] * total_days,
        'order_placed': [False] * total_days,
        'orders_in_transit': [0] * total_days,  # Track orders in transit
        'order_size': [0] * total_days  # Track size of each order placed
    }

    df = pd.DataFrame(data)

    # Set initial inventory, ensuring non-negative stock
    df.at[0, 'stock_start'] = initial_inventory

    # Simulation
    for i in range(total_days):
        if i > 0:
            # Calculate the incoming orders based on lead time
            incoming_orders = df.at[i - lead_time, 'orders_in_transit'] if i - lead_time >= 0 else 0
            df.at[i, 'stock_start'] = df.at[i - 1, 'stock_end'] + incoming_orders
        
        # Calculate end-of-day stock
        df.at[i, 'stock_end'] = df.at[i, 'stock_start'] - df.at[i, 'forecasted_demand']
        
        # Ensure stock_end is not negative
        if df.at[i, 'stock_end'] < 0:
            df.at[i, 'stock_end'] = 0
        
        # Inventory check and order placement
        if (i + 1) % review_period == 0:
            df.at[i, 'inventory_check'] = True
            
            if df.at[i, 'stock_end'] < reorder_point:
                df.at[i, 'order_placed'] = True
                order_size = max_stock_level - df.at[i, 'stock_end']
                df.at[i, 'orders_in_transit'] = order_size  # Track the size of the order placed
                df.at[i, 'order_size'] = order_size

    return df

# Example usage:
if __name__ == "__main__":
    sales_order_file_path = input("Please enter the path to the sales order data file: ")
    result_df = tsl_policy_userinput(sales_order_file_path)
    print(result_df)


Safety Stock: 115.58143714593494
Reorder Point: 598.581437145935
   day_tracker  stock_start  forecasted_demand  stock_end  inventory_check  \
0   2021-04-01       1000.0                 79      921.0            False   
1   2021-04-02        921.0                 37      884.0            False   
2   2021-04-03        884.0                 42      842.0            False   
3   2021-04-04        842.0                 41      801.0            False   
4   2021-04-05        801.0                 44      757.0            False   
..         ...          ...                ...        ...              ...   
95  2021-07-05        993.0                 61      932.0            False   
96  2021-07-06        932.0                134      798.0            False   
97  2021-07-07        798.0                  4      794.0             True   
98  2021-07-08        794.0                 96      698.0            False   
99  2021-07-09        698.0                 74      624.0            False   

/var/folders/ns/1yxzfx892f36qy2_byrlq2kw0000gn/T/ipykernel_71104/2922413859.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  demand_data = pd.read_csv(sales_order_file_path, index_col=0, parse_dates=True)
